In [1]:
import csv
import sys
import pickle
import pandas as pd
import numpy as np
from graphviz import Source
from sklearn import tree
from IPython.display import SVG
from IPython.display import display
from IPython.display import Image
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from PIL import Image
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
import warnings
warnings.filterwarnings('ignore')

In [2]:
def read_dataset(tipo):
    data = pd.read_csv('/home/alison/Documentos/Projeto-Tomate/datasets_generos/dataset_genero_mfcc.csv', sep=',')

    if tipo == 'flor':
        data = data[data['Annotation'] != 'voo']
    elif tipo == 'voo':
        data = data[data['Annotation'] == 'voo']

    data = data.drop(['filename', 'Annotation'],axis=1)

    classes = data.iloc[:, -1]
    standard = StandardScaler()
    matriz = standard.fit_transform(np.array(data.iloc[:, :-1]))
    
    X_train, X_test, y_train, y_test = train_test_split(matriz, classes, test_size=0.5, random_state=0)

    return X_train, X_test, y_train, y_test

In [3]:
def treino_teste(X_train, y_train, modelo, tuned_parameters, filename):
    file_csv = filename + '.csv'
    file_pdf = filename + '.pdf'
    
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

    score = 'f1_macro'

    clf = GridSearchCV(modelo, tuned_parameters, scoring=score, cv=kfold)

    clf.fit(X_train, y_train)

    #y_true = clf.predict(X_test)
    #if score == 'f1_macro':
    #    c, p = y_true, y_pred
    
    #print(metrics.classification_report(y_true, y_pred))
    #cm = pd.crosstab(c, p, rownames=['True'], colnames=['Predicted'], margins=True, dropna=False)
    #cm.to_csv(file_csv, sep='\t')

    #fig = plt.figure(num=None, figsize=(10, 10), dpi=50, facecolor='w', edgecolor='k')
    #tab_acertos = sns.heatmap(cm, cmap='PuRd', annot=True, annot_kws={'size':13}, cbar=False, square=True)
    #tab_acertos.set_xticklabels(tab_acertos.get_xticklabels(), rotation=90) 
    #tab_acertos.get_figure().savefig(file_pdf)
    #plt.show()
    
    return clf.best_params_

## Classificação com SVM

In [4]:
# CLASSIFICAÇÃO SONICATION + FLIGHT
X_train, X_test, y_train, y_test = read_dataset('ambos')

tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-2,1e-3,1e-4], 'C': [0.001, 0.1, 0.01, 1, 10], 'decision_function_shape': ['ovo']},
                    {'kernel': ['poly'], 'gamma': [1e-2,1e-3,1e-4], 'C': [0.001, 0.1, 0.01, 1, 10], 'decision_function_shape': ['ovo']},
                    {'kernel': ['sigmoid'], 'gamma': [1e-2,1e-3,1e-4], 'C': [0.001, 0.1, 0.01, 1, 10], 'decision_function_shape': ['ovo']}, 
                    {'kernel': ['linear'], 'C': [0.001, 0.1, 0.01, 1, 10], 'decision_function_shape': ['ovo']}]

svc_model = SVC()
best_params = treino_teste(X_train, y_train, svc_model, tuned_parameters, 'espcieSVMambos')
print(best_params)

{'C': 10, 'decision_function_shape': 'ovo', 'gamma': 0.01, 'kernel': 'rbf'}


In [5]:
svc_best_model = SVC(C=10, decision_function_shape='ovo', gamma=0.01, kernel='rbf')

svc_best_model.fit(X_train, y_train)

y_pred = svc_best_model.predict(X_test)

print("Acurácia...: %.4f" %(metrics.accuracy_score(y_test, y_pred) * 100))
print("Precision..: %.4f" %(metrics.precision_score(y_test, y_pred, average='macro') * 100))
print("Recall.....: %.4f" %(metrics.recall_score(y_test, y_pred, average='macro') * 100))
print("F1-Score...: %.4f" %(metrics.f1_score(y_test, y_pred, average='macro') * 100))
print()
print(metrics.classification_report(y_test, y_pred))
print()

Acurácia...: 67.9012
Precision..: 57.7927
Recall.....: 60.4630
F1-Score...: 58.5978

                       precision    recall  f1-score   support

       Augochloropsis       0.68      0.62      0.65        21
               Bombus       0.46      0.54      0.50        24
              Centris       0.75      0.75      0.75        12
              Eulaema       0.00      0.00      0.00         1
          Exomalopsis       0.75      0.74      0.74        53
             Melipona       0.79      0.70      0.74        33
Pseudoalglochloropsis       0.55      0.86      0.67         7
             Xylocopa       0.64      0.64      0.64        11

             accuracy                           0.68       162
            macro avg       0.58      0.60      0.59       162
         weighted avg       0.69      0.68      0.68       162




In [6]:
#CLASSIFICAÇÃO SONICATION
X_train, X_test, y_train, y_test = read_dataset('flor')

tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-2,1e-3,1e-4], 'C': [0.001, 0.1, 0.01, 1, 10], 'decision_function_shape': ['ovo']},
                    {'kernel': ['poly'], 'gamma': [1e-2,1e-3,1e-4], 'C': [0.001, 0.1, 0.01, 1, 10], 'decision_function_shape': ['ovo']},
                    {'kernel': ['sigmoid'], 'gamma': [1e-2,1e-3,1e-4], 'C': [0.001, 0.1, 0.01, 1, 10], 'decision_function_shape': ['ovo']}, 
                    {'kernel': ['linear'], 'C': [0.001, 0.1, 0.01, 1, 10], 'decision_function_shape': ['ovo']}]

best_params = treino_teste(X_train, y_train, svc_model, tuned_parameters, 'especieSVMflor')
print(best_params)

{'C': 10, 'decision_function_shape': 'ovo', 'gamma': 0.01, 'kernel': 'rbf'}


In [7]:
svc_best_model = SVC(C=10, decision_function_shape='ovo', gamma=0.01, kernel='rbf')

svc_best_model.fit(X_train, y_train)

y_pred = svc_best_model.predict(X_test)

print("Acurácia...: %.4f" %(metrics.accuracy_score(y_test, y_pred) * 100))
print("Precision..: %.4f" %(metrics.precision_score(y_test, y_pred, average='macro') * 100))
print("Recall.....: %.4f" %(metrics.recall_score(y_test, y_pred, average='macro') * 100))
print("F1-Score...: %.4f" %(metrics.f1_score(y_test, y_pred, average='macro') * 100))
print()
print(metrics.classification_report(y_test, y_pred))
print()

Acurácia...: 66.3636
Precision..: 71.0431
Recall.....: 54.7045
F1-Score...: 58.0624

                       precision    recall  f1-score   support

       Augochloropsis       0.82      0.56      0.67        16
               Bombus       0.47      0.47      0.47        17
              Centris       1.00      0.20      0.33         5
              Eulaema       1.00      1.00      1.00         1
          Exomalopsis       0.74      0.92      0.82        38
             Melipona       0.65      0.72      0.68        18
Pseudoalglochloropsis       1.00      0.50      0.67        12
             Xylocopa       0.00      0.00      0.00         3

             accuracy                           0.66       110
            macro avg       0.71      0.55      0.58       110
         weighted avg       0.72      0.66      0.66       110




In [8]:
#CLASSIFICAÇÃO FLIGHT
X_train, X_test, y_train, y_test = read_dataset('voo')

tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-2,1e-3,1e-4], 'C': [0.001, 0.1, 0.01, 1, 10], 'decision_function_shape': ['ovo']},
                    {'kernel': ['poly'], 'gamma': [1e-2,1e-3,1e-4], 'C': [0.001, 0.1, 0.01, 1, 10], 'decision_function_shape': ['ovo']},
                    {'kernel': ['sigmoid'], 'gamma': [1e-2,1e-3,1e-4], 'C': [0.001, 0.1, 0.01, 1, 10], 'decision_function_shape': ['ovo']}, 
                    {'kernel': ['linear'], 'C': [0.001, 0.1, 0.01, 1, 10], 'decision_function_shape': ['ovo']}]

best_params = treino_teste(X_train, y_train, svc_model, tuned_parameters, 'especieSVMvoo')
print(best_params)

{'C': 10, 'decision_function_shape': 'ovo', 'gamma': 0.01, 'kernel': 'rbf'}


In [9]:
svc_best_model = SVC(C=10, decision_function_shape='ovo', gamma=0.01, kernel='rbf')

svc_best_model.fit(X_train, y_train)

y_pred = svc_best_model.predict(X_test)

print("Acurácia...: %.4f" %(metrics.accuracy_score(y_test, y_pred) * 100))
print("Precision..: %.4f" %(metrics.precision_score(y_test, y_pred, average='macro') * 100))
print("Recall.....: %.4f" %(metrics.recall_score(y_test, y_pred, average='macro') * 100))
print("F1-Score...: %.4f" %(metrics.f1_score(y_test, y_pred, average='macro') * 100))
print()
print(metrics.classification_report(y_test, y_pred))
print()

Acurácia...: 64.1509
Precision..: 64.4426
Recall.....: 60.4911
F1-Score...: 60.2068

                       precision    recall  f1-score   support

       Augochloropsis       1.00      0.38      0.55         8
               Bombus       0.71      1.00      0.83        10
              Centris       0.83      0.71      0.77         7
              Eulaema       1.00      1.00      1.00         1
          Exomalopsis       0.40      0.50      0.44         8
             Melipona       0.64      0.58      0.61        12
Pseudoalglochloropsis       0.00      0.00      0.00         1
             Xylocopa       0.57      0.67      0.62         6

             accuracy                           0.64        53
            macro avg       0.64      0.60      0.60        53
         weighted avg       0.68      0.64      0.63        53




## Classificação LR

In [10]:
#CLASSIFICAÇÃO SONICATION + FLIGHT
X_train, X_test, y_train, y_test = read_dataset('ambos')

tuned_parameters = [{'penalty': ['l1', 'l2']},
                    {'C': [0.001, 0.1, 1, 10, 100]}]

lr_model = LogisticRegression()
best_params = treino_teste(X_train, y_train, lr_model, tuned_parameters, 'especieLRambos')
print(best_params)

{'penalty': 'l2'}


In [12]:
lr_best_model = LogisticRegression(penalty='l2')

lr_best_model.fit(X_train, y_train)

y_pred = lr_best_model.predict(X_test)

print("Acurácia...: %.4f" %(metrics.accuracy_score(y_test, y_pred) * 100))
print("Precision..: %.4f" %(metrics.precision_score(y_test, y_pred, average='macro') * 100))
print("Recall.....: %.4f" %(metrics.recall_score(y_test, y_pred, average='macro') * 100))
print("F1-Score...: %.4f" %(metrics.f1_score(y_test, y_pred, average='macro') * 100))
print()
print(metrics.classification_report(y_test, y_pred))
print()

Acurácia...: 62.3457
Precision..: 51.4969
Recall.....: 55.7696
F1-Score...: 52.3808

                       precision    recall  f1-score   support

       Augochloropsis       0.73      0.52      0.61        21
               Bombus       0.38      0.25      0.30        24
              Centris       0.69      0.75      0.72        12
              Eulaema       0.00      0.00      0.00         1
          Exomalopsis       0.69      0.72      0.70        53
             Melipona       0.67      0.73      0.70        33
Pseudoalglochloropsis       0.46      0.86      0.60         7
             Xylocopa       0.50      0.64      0.56        11

             accuracy                           0.62       162
            macro avg       0.51      0.56      0.52       162
         weighted avg       0.62      0.62      0.61       162




In [13]:
#CLASSIFICAÇÃO SONICATION
X_train, X_test, y_train, y_test = read_dataset('flor')

tuned_parameters = [{'penalty': ['l1', 'l2']},
                    {'C': [0.001, 0.1, 1, 10, 100]}]

lr_model = LogisticRegression()
best_params = treino_teste(X_train, y_train, lr_model, tuned_parameters, 'especieLRflor')
print(best_params)

{'penalty': 'l2'}


In [14]:
lr_best_model = LogisticRegression(penalty='l2')

lr_best_model.fit(X_train, y_train)

y_pred = lr_best_model.predict(X_test)

print("Acurácia...: %.4f" %(metrics.accuracy_score(y_test, y_pred) * 100))
print("Precision..: %.4f" %(metrics.precision_score(y_test, y_pred, average='macro') * 100))
print("Recall.....: %.4f" %(metrics.recall_score(y_test, y_pred, average='macro') * 100))
print("F1-Score...: %.4f" %(metrics.f1_score(y_test, y_pred, average='macro') * 100))
print()
print(metrics.classification_report(y_test, y_pred))
print()

Acurácia...: 60.9091
Precision..: 56.7106
Recall.....: 49.9156
F1-Score...: 51.5538

                       precision    recall  f1-score   support

       Augochloropsis       0.86      0.38      0.52        16
               Bombus       0.40      0.47      0.43        17
              Centris       0.00      0.00      0.00         5
              Eulaema       1.00      1.00      1.00         1
          Exomalopsis       0.64      0.84      0.73        38
             Melipona       0.76      0.72      0.74        18
Pseudoalglochloropsis       0.88      0.58      0.70        12
             Xylocopa       0.00      0.00      0.00         3

             accuracy                           0.61       110
            macro avg       0.57      0.50      0.52       110
         weighted avg       0.64      0.61      0.60       110




In [15]:
#CLASSIFICAÇÃO SONICATION + FLIGHT
X_train, X_test, y_train, y_test = read_dataset('voo')

tuned_parameters = [{'penalty': ['l1', 'l2']},
                    {'C': [0.001, 0.1, 1, 10, 100]}]

lr_model = LogisticRegression()
best_params = treino_teste(X_train, y_train, lr_model, tuned_parameters, 'especieLRvoo')
print(best_params)

{'penalty': 'l2'}


In [16]:
lr_best_model = LogisticRegression(penalty='l2')

lr_best_model.fit(X_train, y_train)

y_pred = lr_best_model.predict(X_test)

print("Acurácia...: %.4f" %(metrics.accuracy_score(y_test, y_pred) * 100))
print("Precision..: %.4f" %(metrics.precision_score(y_test, y_pred, average='macro') * 100))
print("Recall.....: %.4f" %(metrics.recall_score(y_test, y_pred, average='macro') * 100))
print("F1-Score...: %.4f" %(metrics.f1_score(y_test, y_pred, average='macro') * 100))
print()
print(metrics.classification_report(y_test, y_pred))
print()

Acurácia...: 60.3774
Precision..: 65.2540
Recall.....: 56.6369
F1-Score...: 57.0270

                       precision    recall  f1-score   support

       Augochloropsis       1.00      0.25      0.40         8
               Bombus       0.53      0.90      0.67        10
              Centris       1.00      0.71      0.83         7
              Eulaema       1.00      1.00      1.00         1
          Exomalopsis       0.36      0.50      0.42         8
             Melipona       0.73      0.67      0.70        12
Pseudoalglochloropsis       0.00      0.00      0.00         1
             Xylocopa       0.60      0.50      0.55         6

             accuracy                           0.60        53
            macro avg       0.65      0.57      0.57        53
         weighted avg       0.69      0.60      0.60        53




## Classificação RF

In [17]:
#CLASSIFICAÇÃO SONICATION + FLIGHT
X_train, X_test, y_train, y_test = read_dataset('ambos')

tuned_parameters = {'n_estimators': [100, 200],
                    'max_features': ['auto', 'sqrt', 'log2']}

rf_model = RandomForestClassifier()
best_param = treino_teste(X_train, y_train, rf_model, tuned_parameters, 'especieRFambos')
print(best_param)

{'max_features': 'sqrt', 'n_estimators': 100}


In [19]:
rf_best_model = RandomForestClassifier(max_features='auto', n_estimators=100)

rf_best_model.fit(X_train, y_train)

y_pred = rf_best_model.predict(X_test)

print("Acurácia...: %.4f" %(metrics.accuracy_score(y_test, y_pred) * 100))
print("Precision..: %.4f" %(metrics.precision_score(y_test, y_pred, average='macro') * 100))
print("Recall.....: %.4f" %(metrics.recall_score(y_test, y_pred, average='macro') * 100))
print("F1-Score...: %.4f" %(metrics.f1_score(y_test, y_pred, average='macro') * 100))
print()
print(metrics.classification_report(y_test, y_pred))
print()

Acurácia...: 61.1111
Precision..: 52.4607
Recall.....: 47.9576
F1-Score...: 48.3123

                       precision    recall  f1-score   support

       Augochloropsis       0.75      0.43      0.55        21
               Bombus       0.35      0.29      0.32        24
              Centris       0.88      0.58      0.70        12
              Eulaema       0.00      0.00      0.00         1
          Exomalopsis       0.69      0.85      0.76        53
             Melipona       0.57      0.70      0.63        33
Pseudoalglochloropsis       0.45      0.71      0.56         7
             Xylocopa       0.50      0.27      0.35        11

             accuracy                           0.61       162
            macro avg       0.52      0.48      0.48       162
         weighted avg       0.61      0.61      0.60       162




In [20]:
#CLASSIFICAÇÃO SONICATION
X_train, X_test, y_train, y_test = read_dataset('flor')

tuned_parameters = {'n_estimators': [100, 200],
                    'max_features': ['auto', 'sqrt', 'log2']}

rf_model = RandomForestClassifier()
best_param = treino_teste(X_train, y_train, rf_model, tuned_parameters, 'especieRFflor')
print(best_param)

{'max_features': 'auto', 'n_estimators': 200}


In [23]:
rf_best_model = RandomForestClassifier(max_features='auto', n_estimators=100)

rf_best_model.fit(X_train, y_train)

y_pred = rf_best_model.predict(X_test)

print("Acurácia...: %.4f" %(metrics.accuracy_score(y_test, y_pred) * 100))
print("Precision..: %.4f" %(metrics.precision_score(y_test, y_pred, average='macro') * 100))
print("Recall.....: %.4f" %(metrics.recall_score(y_test, y_pred, average='macro') * 100))
print("F1-Score...: %.4f" %(metrics.f1_score(y_test, y_pred, average='macro') * 100))
print()
print(metrics.classification_report(y_test, y_pred))
print()

Acurácia...: 62.7273
Precision..: 46.1992
Recall.....: 36.9362
F1-Score...: 38.2138

                       precision    recall  f1-score   support

       Augochloropsis       1.00      0.38      0.55        16
               Bombus       0.54      0.41      0.47        17
              Centris       0.00      0.00      0.00         5
              Eulaema       0.00      0.00      0.00         1
          Exomalopsis       0.59      0.97      0.73        38
             Melipona       0.74      0.78      0.76        18
Pseudoalglochloropsis       0.83      0.42      0.56        12
             Xylocopa       0.00      0.00      0.00         3

             accuracy                           0.63       110
            macro avg       0.46      0.37      0.38       110
         weighted avg       0.64      0.63      0.59       110




In [24]:
#CLASSIFICAÇÃO FLIGHT
X_train, X_test, y_train, y_test = read_dataset('voo')

tuned_parameters = {'n_estimators': [100, 200],
                    'max_features': ['auto', 'sqrt', 'log2']}

rf_model = RandomForestClassifier()
best_param = treino_teste(X_train, y_train, rf_model, tuned_parameters, 'especieRFvoo')
print(best_param)

{'max_features': 'log2', 'n_estimators': 200}


In [31]:
rf_best_model = RandomForestClassifier(max_features='log2', n_estimators=200)

rf_best_model.fit(X_train, y_train)

y_pred = rf_best_model.predict(X_test)

print("Acurácia...: %.4f" %(metrics.accuracy_score(y_test, y_pred) * 100))
print("Precision..: %.4f" %(metrics.precision_score(y_test, y_pred, average='macro') * 100))
print("Recall.....: %.4f" %(metrics.recall_score(y_test, y_pred, average='macro') * 100))
print("F1-Score...: %.4f" %(metrics.f1_score(y_test, y_pred, average='macro') * 100))
print()
print(metrics.classification_report(y_test, y_pred))
print()

Acurácia...: 56.6038
Precision..: 43.4926
Recall.....: 42.5744
F1-Score...: 41.3199

                       precision    recall  f1-score   support

       Augochloropsis       0.50      0.25      0.33         8
               Bombus       0.53      0.90      0.67        10
              Centris       0.83      0.71      0.77         7
              Eulaema       0.00      0.00      0.00         1
          Exomalopsis       0.42      0.62      0.50         8
             Melipona       0.70      0.58      0.64        12
Pseudoalglochloropsis       0.00      0.00      0.00         1
             Xylocopa       0.50      0.33      0.40         6

             accuracy                           0.57        53
            macro avg       0.43      0.43      0.41        53
         weighted avg       0.56      0.57      0.54        53




## Classificação DTree

In [32]:
#CLASSIFICAÇÃO SONICATION + FLIGHT
X_train, X_test, y_train, y_test = read_dataset('ambos')

tuned_parameters = {"criterion": ["gini", "entropy"],
                    "min_samples_split": [2, 10],
                    "max_depth": [2, 5, 10]
                    }

detree_model = DecisionTreeClassifier()
best_param = treino_teste(X_train, y_train, detree_model, tuned_parameters, 'especieDTREEambos')
print(best_param)

{'criterion': 'entropy', 'max_depth': 5, 'min_samples_split': 2}


In [35]:
dtree_best_model = DecisionTreeClassifier(criterion='entropy', max_depth=5, min_samples_split=2)

dtree_best_model.fit(X_train, y_train)

y_pred = dtree_best_model.predict(X_test)

print("Acurácia...: %.4f" %(metrics.accuracy_score(y_test, y_pred) * 100))
print("Precision..: %.4f" %(metrics.precision_score(y_test, y_pred, average='macro') * 100))
print("Recall.....: %.4f" %(metrics.recall_score(y_test, y_pred, average='macro') * 100))
print("F1-Score...: %.4f" %(metrics.f1_score(y_test, y_pred, average='macro') * 100))
print()
print(metrics.classification_report(y_test, y_pred))
print()

Acurácia...: 39.5062
Precision..: 28.9132
Recall.....: 31.6714
F1-Score...: 28.5297

                       precision    recall  f1-score   support

       Augochloropsis       0.42      0.38      0.40        21
               Bombus       0.23      0.42      0.29        24
              Centris       0.24      0.33      0.28        12
              Eulaema       0.00      0.00      0.00         1
          Exomalopsis       0.72      0.53      0.61        53
             Melipona       0.48      0.30      0.37        33
Pseudoalglochloropsis       0.24      0.57      0.33         7
             Xylocopa       0.00      0.00      0.00        11

             accuracy                           0.40       162
            macro avg       0.29      0.32      0.29       162
         weighted avg       0.45      0.40      0.40       162




In [36]:
#CLASSIFICAÇÃO SONICATION
X_train, X_test, y_train, y_test = read_dataset('flor')

tuned_parameters = {"criterion": ["gini", "entropy"],
                    "min_samples_split": [2, 10],
                    "max_depth": [2, 5, 10]
                    }

detree_model = DecisionTreeClassifier()
best_param = treino_teste(X_train, y_train, detree_model, tuned_parameters, 'especieDTREEflor')
print(best_param)

{'criterion': 'entropy', 'max_depth': 5, 'min_samples_split': 2}


In [37]:
dtree_best_model = DecisionTreeClassifier(criterion='entropy', max_depth=5, min_samples_split=2)

dtree_best_model.fit(X_train, y_train)

y_pred = dtree_best_model.predict(X_test)

print("Acurácia...: %.4f" %(metrics.accuracy_score(y_test, y_pred) * 100))
print("Precision..: %.4f" %(metrics.precision_score(y_test, y_pred, average='macro') * 100))
print("Recall.....: %.4f" %(metrics.recall_score(y_test, y_pred, average='macro') * 100))
print("F1-Score...: %.4f" %(metrics.f1_score(y_test, y_pred, average='macro') * 100))
print()
print(metrics.classification_report(y_test, y_pred))
print()

Acurácia...: 43.6364
Precision..: 25.2731
Recall.....: 25.5840
F1-Score...: 25.2677

                       precision    recall  f1-score   support

       Augochloropsis       0.27      0.19      0.22        16
               Bombus       0.17      0.18      0.17        17
              Centris       0.00      0.00      0.00         5
              Eulaema       0.00      0.00      0.00         1
          Exomalopsis       0.63      0.71      0.67        38
             Melipona       0.50      0.56      0.53        18
Pseudoalglochloropsis       0.45      0.42      0.43        12
             Xylocopa       0.00      0.00      0.00         3

             accuracy                           0.44       110
            macro avg       0.25      0.26      0.25       110
         weighted avg       0.41      0.44      0.42       110




In [38]:
#CLASSIFICAÇÃO FLIGHT
X_train, X_test, y_train, y_test = read_dataset('voo')

tuned_parameters = {"criterion": ["gini", "entropy"],
                    "min_samples_split": [2, 10],
                    "max_depth": [2, 5, 10]
                    }

detree_model = DecisionTreeClassifier()
best_param = treino_teste(X_train, y_train, detree_model, tuned_parameters, 'especieDTREEvoo')
print(best_param)

{'criterion': 'gini', 'max_depth': 2, 'min_samples_split': 2}


In [41]:
dtree_best_model = DecisionTreeClassifier(criterion='gini', max_depth=2, min_samples_split=10)

dtree_best_model.fit(X_train, y_train)

y_pred = dtree_best_model.predict(X_test)

print("Acurácia...: %.4f" %(metrics.accuracy_score(y_test, y_pred) * 100))
print("Precision..: %.4f" %(metrics.precision_score(y_test, y_pred, average='macro') * 100))
print("Recall.....: %.4f" %(metrics.recall_score(y_test, y_pred, average='macro') * 100))
print("F1-Score...: %.4f" %(metrics.f1_score(y_test, y_pred, average='macro') * 100))
print()
print(metrics.classification_report(y_test, y_pred))
print()

Acurácia...: 39.6226
Precision..: 20.9226
Recall.....: 28.7946
F1-Score...: 21.8583

                       precision    recall  f1-score   support

       Augochloropsis       0.00      0.00      0.00         8
               Bombus       0.42      1.00      0.59        10
              Centris       0.30      0.43      0.35         7
              Eulaema       0.00      0.00      0.00         1
          Exomalopsis       0.36      0.62      0.45         8
             Melipona       0.60      0.25      0.35        12
Pseudoalglochloropsis       0.00      0.00      0.00         1
             Xylocopa       0.00      0.00      0.00         6

             accuracy                           0.40        53
            macro avg       0.21      0.29      0.22        53
         weighted avg       0.31      0.40      0.31        53




## Classificação Ensemble

In [42]:
#CLASSIFICAÇÃO SONICATION + FLIGHT
X_train, X_test, y_train, y_test = read_dataset('ambos')

tuned_parameters = {'lr__C': [0.001, 0.1, 1, 10, 100],
                    'svc__C': [0.001, 0.1, 0.01, 1, 10]}

svc = SVC()
rf = RandomForestClassifier()
lr = LogisticRegression()

modelos = [('svc', svc), ('rf', rf), ('lr', lr)]

votingclf = VotingClassifier(estimators=modelos, voting='hard')
best_param = treino_teste(X_train, y_train, votingclf, tuned_parameters, 'especieENSEMBLEambos')
print(best_param)

{'lr__C': 100, 'svc__C': 10}


In [45]:
best_svc = SVC(C=10)
rf = RandomForestClassifier()
best_lr = LogisticRegression(C=1)

modelos = [('svc', best_svc), ('rf', rf), ('lr', best_lr)]

best_votingclf = VotingClassifier(estimators=modelos, voting='hard')

best_votingclf.fit(X_train, y_train)

y_pred = best_votingclf.predict(X_test)

print("Acurácia...: %.4f" %(metrics.accuracy_score(y_test, y_pred) * 100))
print("Precision..: %.4f" %(metrics.precision_score(y_test, y_pred, average='macro') * 100))
print("Recall.....: %.4f" %(metrics.recall_score(y_test, y_pred, average='macro') * 100))
print("F1-Score...: %.4f" %(metrics.f1_score(y_test, y_pred, average='macro') * 100))
print()
print(metrics.classification_report(y_test, y_pred))
print()

Acurácia...: 68.5185
Precision..: 60.5691
Recall.....: 55.5884
F1-Score...: 56.7396

                       precision    recall  f1-score   support

       Augochloropsis       0.73      0.52      0.61        21
               Bombus       0.45      0.58      0.51        24
              Centris       0.78      0.58      0.67        12
              Eulaema       0.00      0.00      0.00         1
          Exomalopsis       0.76      0.83      0.79        53
             Melipona       0.74      0.76      0.75        33
Pseudoalglochloropsis       0.56      0.71      0.63         7
             Xylocopa       0.83      0.45      0.59        11

             accuracy                           0.69       162
            macro avg       0.61      0.56      0.57       162
         weighted avg       0.70      0.69      0.68       162




In [46]:
#CLASSIFICAÇÃO SONICATION
X_train, X_test, y_train, y_test = read_dataset('flor')

tuned_parameters = {'lr__C': [0.001, 0.1, 1, 10, 100],
                    'svc__C': [0.001, 0.1, 0.01, 1, 10]}

svc = SVC()
rf = RandomForestClassifier()
lr = LogisticRegression()

modelos = [('svc', svc), ('rf', rf), ('lr', lr)]

votingclf = VotingClassifier(estimators=modelos, voting='hard')
best_param = treino_teste(X_train, y_train, votingclf, tuned_parameters, 'especieENSEMBLEflor')
print(best_param)

{'lr__C': 10, 'svc__C': 10}


In [47]:
best_svc = SVC(C=10)
rf = RandomForestClassifier()
best_lr = LogisticRegression(C=10)

modelos = [('svc', best_svc), ('rf', rf), ('lr', best_lr)]

best_votingclf = VotingClassifier(estimators=modelos, voting='hard')

best_votingclf.fit(X_train, y_train)

y_pred = best_votingclf.predict(X_test)

print("Acurácia...: %.4f" %(metrics.accuracy_score(y_test, y_pred) * 100))
print("Precision..: %.4f" %(metrics.precision_score(y_test, y_pred, average='macro') * 100))
print("Recall.....: %.4f" %(metrics.recall_score(y_test, y_pred, average='macro') * 100))
print("F1-Score...: %.4f" %(metrics.f1_score(y_test, y_pred, average='macro') * 100))
print()
print(metrics.classification_report(y_test, y_pred))
print()

Acurácia...: 67.2727
Precision..: 47.8655
Recall.....: 41.9104
F1-Score...: 42.9794

                       precision    recall  f1-score   support

       Augochloropsis       0.90      0.56      0.69        16
               Bombus       0.48      0.65      0.55        17
              Centris       0.00      0.00      0.00         5
              Eulaema       0.00      0.00      0.00         1
          Exomalopsis       0.69      0.92      0.79        38
             Melipona       0.76      0.72      0.74        18
Pseudoalglochloropsis       1.00      0.50      0.67        12
             Xylocopa       0.00      0.00      0.00         3

             accuracy                           0.67       110
            macro avg       0.48      0.42      0.43       110
         weighted avg       0.68      0.67      0.65       110




In [48]:
#CLASSIFICAÇÃO FLIGHT
X_train, X_test, y_train, y_test = read_dataset('voo')

tuned_parameters = {'lr__C': [0.001, 0.1, 1, 10, 100],
                    'svc__C': [0.001, 0.1, 0.01, 1, 10]}

svc = SVC()
rf = RandomForestClassifier()
lr = LogisticRegression()

modelos = [('svc', svc), ('rf', rf), ('lr', lr)]

votingclf = VotingClassifier(estimators=modelos, voting='hard')
best_param = treino_teste(X_train, y_train, votingclf, tuned_parameters, 'especieENSEMBLEvoo')
print(best_param)

{'lr__C': 100, 'svc__C': 10}


In [50]:
best_svc = SVC(C=10)
rf = RandomForestClassifier()
best_lr = LogisticRegression(C=10)

modelos = [('svc', best_svc), ('rf', rf), ('lr', best_lr)]

best_votingclf = VotingClassifier(estimators=modelos, voting='hard')

best_votingclf.fit(X_train, y_train)

y_pred = best_votingclf.predict(X_test)

print("Acurácia...: %.4f" %(metrics.accuracy_score(y_test, y_pred) * 100))
print("Precision..: %.4f" %(metrics.precision_score(y_test, y_pred, average='macro') * 100))
print("Recall.....: %.4f" %(metrics.recall_score(y_test, y_pred, average='macro') * 100))
print("F1-Score...: %.4f" %(metrics.f1_score(y_test, y_pred, average='macro') * 100))
print()
print(metrics.classification_report(y_test, y_pred))
print()

Acurácia...: 60.3774
Precision..: 60.3706
Recall.....: 56.8452
F1-Score...: 56.3348

                       precision    recall  f1-score   support

       Augochloropsis       0.67      0.25      0.36         8
               Bombus       0.59      1.00      0.74        10
              Centris       0.83      0.71      0.77         7
              Eulaema       1.00      1.00      1.00         1
          Exomalopsis       0.36      0.50      0.42         8
             Melipona       0.78      0.58      0.67        12
Pseudoalglochloropsis       0.00      0.00      0.00         1
             Xylocopa       0.60      0.50      0.55         6

             accuracy                           0.60        53
            macro avg       0.60      0.57      0.56        53
         weighted avg       0.64      0.60      0.59        53


